<a href="https://colab.research.google.com/github/dipta007/Movie-Recommendation/blob/one-hot-encoding/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from os.path import join
import os

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/Colab_data/MR1'       # path to your project on Drive
drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   
%cd "{PROJECT_PATH}"
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Colab_data/MR1’: File exists
/content/drive/My Drive/Colab_data/MR1
data/


In [2]:
%matplotlib inline
import numpy as np
import pandas as pd

DATA_PATH = './data/ml-latest-small'

ratings = pd.read_csv(f"{DATA_PATH}/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies = pd.read_csv(f"{DATA_PATH}/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

user_enc = LabelEncoder()
ratings['user'] = user_enc.fit_transform(ratings['userId'].values)
n_users = ratings['user'].nunique()

movie_enc = LabelEncoder()
ratings['movie'] = movie_enc.fit_transform(ratings['movieId'].values)
n_movies = ratings['movie'].nunique()

ratings['rating'] = ratings['rating'].values.astype(np.float32)

scaler = MinMaxScaler()
scaler.fit(ratings['rating'].values.reshape(-1, 1))
ratings['normalized_rating'] = scaler.transform(ratings['rating'].values.reshape(-1, 1))
min_rating = min(ratings['rating'])
max_rating = max(ratings['rating'])

# n_users, n_movies, min_rating, max_rating
# del ratings['timestamp']
ratings.head()

,userId,movieId,rating,timestamp,user,movie,normalized_rating
0,1,1,4.0,964982703,0,0,0.777778
1,1,3,4.0,964981247,0,2,0.777778
2,1,6,4.0,964982224,0,5,0.777778
3,1,47,5.0,964983815,0,43,1.000000
4,1,50,5.0,964982931,0,46,1.000000


In [0]:
user_movie_count = [0 for _ in range(n_users)]
user_movie = {}
user_movie_rating = {}
for index, row in ratings.iterrows():
  now = int(row.user)
  user_movie_count[now] += 1
  if now not in user_movie:
    user_movie[now] = []
    user_movie_rating[now] = []
  user_movie[now].append(row.movie)
  user_movie_rating[now].append(row.normalized_rating)

MIN_NUMBER_OF_MOVIES = min(min(user_movie_count), 4)

In [6]:
MIN_NUMBER_OF_MOVIES, n_users, n_movies, min_rating, max_rating

(4, 610, 9724, 0.5, 5.0)

In [0]:
def convert_to_one_hot(movies, ratings):
  now = [0 for _ in range(n_movies)]
  for i in range(len(movies)):
    now_ind = int(movies[i])
    now_rating = ratings[i]
    now[now_ind] = now_rating

  return now

prev_movies = []
next_movies = []
for user in range(n_users):
  for i in range(len(user_movie[user]) - MIN_NUMBER_OF_MOVIES):
    now_movies = user_movie[user]
    now_ratings = user_movie_rating[user]
    
    # Previous movies he has saw already
    prev_movies.append(convert_to_one_hot(now_movies[i:i+MIN_NUMBER_OF_MOVIES],
                                          now_ratings[i:i+MIN_NUMBER_OF_MOVIES]));

    # Next movies he should see
    next_movies.append(convert_to_one_hot(now_movies[i+MIN_NUMBER_OF_MOVIES:],
                                          now_ratings[i+MIN_NUMBER_OF_MOVIES:]));


In [8]:
len(prev_movies), len(next_movies)

(98396, 98396)

In [14]:
len(prev_movies[0])

9724

In [0]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2
from keras.layers import Concatenate, Dense, Dropout, Activation, Lambda, LSTM

def Model1(n_users, n_movies):
  input1 = Input(shape=(n_movies,))
  x = LSTM(128)(input1)
  x = Dense(n_movies, activation="sigmoid")(x)

  model = Model(inputs=input1, outputs=x)
  rms = RMSprop(lr=0.001)
  model.compile(loss='mean_squared_error', optimizer=rms)
  return model

In [44]:
model = Model1(n_users, n_movies)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 1, 9724)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)               5044736   
_________________________________________________________________
dense_2 (Dense)              (None, 9724)              1254396   
Total params: 6,299,132
Trainable params: 6,299,132
Non-trainable params: 0
_________________________________________________________________


In [45]:
history = model.fit(x=prev_movies, y=next_movies, batch_size=64, epochs=5, 
                    verbose=1)

ValueError: ignored

In [9]:
X = ratings[['user', 'movie']].values
y = ratings['rating'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=47)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((90752, 2), (10084, 2), (90752,), (10084,))

In [0]:
n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [0]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import Concatenate, Dense, Dropout, Activation, Lambda

def RecommenderV1(n_users, n_movies, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal', embeddings_regularizer=l2(1e-6))(user)
    u = Reshape((n_factors,))(u)
    
    movie = Input(shape=(1,))
    m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal', embeddings_regularizer=l2(1e-6))(movie)
    m = Reshape((n_factors,))(m)
    
    x = Concatenate()([u, m])
    x = Dropout(0.2)(x)

    x = Dense(100, kernel_initializer='he_normal', activation="relu")(x)
    x = Dropout(0.2)(x)

    x = Dense(50, kernel_initializer='he_normal', activation="relu")(x)
    x = Dropout(0.2)(x)

    x = Dense(20, kernel_initializer='he_normal', activation="relu")(x)
    x = Dropout(0.2)(x)

    x = Dense(10, kernel_initializer='he_normal', activation="relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(1, kernel_initializer='he_normal', activation="sigmoid")(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)

    model = Model(inputs=[user, movie], outputs=x)
    adam = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=adam)
    return model

In [29]:
model = RecommenderV1(n_users, n_movies, n_factors)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 1, 50)        30500       input_9[0][0]                    
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 1, 50)        486200      input_10[0][0]                   
____________________________________________________________________________________________

In [30]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5, 
                    verbose=1, validation_data=(X_test_array, y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 90752 samples, validate on 10084 samples
Epoch 1/5
90752/90752 [==============================] - 12s 134us/step - loss: 0.8969 - val_loss: 0.7520
Epoch 2/5
90752/90752 [==============================] - 12s 129us/step - loss: 0.7486 - val_loss: 0.7314
Epoch 3/5
90752/90752 [==============================] - 12s 133us/step - loss: 0.7005 - val_loss: 0.7349
Epoch 4/5
90752/90752 [==============================] - 12s 129us/step - loss: 0.6697 - val_loss: 0.7289
Epoch 5/5
90752/90752 [==============================] - 12s 134us/step - loss: 0.6431 - val_loss: 0.7438


In [0]:
def 